In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("data/processed/train_df_fold.csv")
df_input = pd.read_csv("data/input/train_records.csv")

In [3]:
df_list = []

# df_inputを1行ずつ見ていく
for i in range(len(df_input)):
    check_id = df_input["id"].iloc[i]
    # df_trainのidがcheck_idと一致する行を抽出
    df_train_id = df_train.loc[df_train["id"] == check_id].copy()
    if len(df_train_id) == 0:
        continue
    df_train_id["night"] = df_input["night"][i]
    df_train_id["age"] = df_input["age"][i]
    df_train_id["sex"] = df_input["sex"][i]

    # df_train_idに消灯時間からの時間を追加
    # df_inputのlights_offとdf_train_idのmeas_timeの差分を取る
    df_input["lights_off"] = pd.to_datetime(df_input["lights_off"])
    df_train_id["meas_time"] = pd.to_datetime(df_train_id["meas_time"])
    # lights_offの年月をmeas_timeの年月に合わせる
    y, m, d = df_train_id["meas_time"].iloc[0].year, df_train_id["meas_time"].iloc[0].month, df_train_id["meas_time"].iloc[0].day
    df_train_id["sleep_duration"] = (df_train_id["meas_time"] - df_train_id["meas_time"].iloc[0]).dt.total_seconds() / 60

    # df_train_idに一個前のラベルを追加
    df_train_id["label_before"] = df_train_id["label"].shift(1)

    df_list.append(df_train_id)

# 結合
df_train_with_meta = pd.concat(df_list)

# label_beforeの欠損値をその行のlabelに置き換える
df_train_with_meta["label_before"] = df_train_with_meta["label_before"].fillna(df_train_with_meta["label"])

print(df_train_with_meta.head())
print(df_train.head())
# sleep_durationの最小値を取得
print(df_train_with_meta["sleep_duration"].min())
print(df_train_with_meta["sleep_duration"].max())

   epoch           meas_time  label       id  fold  night  age   sex  \
0      0 1989-11-13 21:35:00      0  3c1c5cf     1      1   90  male   
1      1 1989-11-13 21:35:30      0  3c1c5cf     1      1   90  male   
2      2 1989-11-13 21:36:00      0  3c1c5cf     1      1   90  male   
3      3 1989-11-13 21:36:30      0  3c1c5cf     1      1   90  male   
4      4 1989-11-13 21:37:00      0  3c1c5cf     1      1   90  male   

   sleep_duration  label_before  
0             0.0           0.0  
1             0.5           0.0  
2             1.0           0.0  
3             1.5           0.0  
4             2.0           0.0  
   epoch            meas_time  label       id  fold
0      0  1989-11-13 21:35:00      0  3c1c5cf     1
1      1  1989-11-13 21:35:30      0  3c1c5cf     1
2      2  1989-11-13 21:36:00      0  3c1c5cf     1
3      3  1989-11-13 21:36:30      0  3c1c5cf     1
4      4  1989-11-13 21:37:00      0  3c1c5cf     1
0.0
839.5


In [4]:
# df_trainでepochが0の行のlabelを取得して、各ラベルの数をカウント
print(df_train.loc[df_train["epoch"] == 0]["label"].value_counts())
print(df_train_with_meta.loc[df_train_with_meta["epoch"] == 0]["label"].value_counts())

0    106
2      2
Name: label, dtype: int64
0    106
2      2
Name: label, dtype: int64


In [5]:
# df_list内のDataframeのindexが連続しているか確認
idx_tmp = -1
for df_i in range(len(df_list)):
    for idx in df_list[df_i].index:
        if idx_tmp + 1 != idx:
            print(df_i, idx)
        idx_tmp = idx

In [6]:
print(len(df_train_with_meta))
print(len(df_train))

161610
161610
